# Лабораторная работа. Ранжирование.

![](http://i.imgur.com/2QnD2nF.jpg)

Задачу поискового ранжирования можно описать следующим образом: имеется множество документов $d \in D$ и множество запросов $q \in Q$. Требуется оценить *степень релевантности* документа по отношению к запросу: $(q, d) \mapsto r$, относительно которой будет производиться ранжирование. Для восстановления этой зависимости используются методы машинного обучения.     
Обычно используется три типа признаков:
 - признаки запроса $q$, например: мешок слов текста запроса, его длина, ...
 - документа $d$, например: значение PageRank, мешок слов, доменное имя, ...
 - пары $(q, d)$, например: число вхождений фразы из запроса $q$ в документе $d$, ...

Одна из отличительных особенностей задачи ранжирования от классических задач машинного обучения заключается в том, что качество результата зависит не от предсказанных оценок релевантности, а от порядка следования документов в рамках конкретного запроса, т.е. важно не абсолютное значение релевантности (его достаточно трудно формализовать в виде числа), а то, более или менее релевантен документ, относительно других документов.
### Подходы к решению задачи ранжирования
Существуют 3 основных подхода к ранжированию, различие между которыми заключается в том, на какую функцию потерь они опираются:
  
1. **Поточечный подход (pointwise)**. В этом подходе предполагается, что каждой паре запрос-документ поставлена в соответствие численная оценка. Задача обучения ранжированию сводится к построению регрессии: для каждой отдельной пары запрос-документ необходимо предсказать её оценку.

2. **Попарный подход (pairwise)**. В таком подходе обучение ранжированию сводится к построению бинарного классификатора, которому на вход поступают два документа, соответствующих одному и тому же запросу, и требуется определить, какой из них лучше. Другими словами, функция потерь штрафует модель, если отранжированная этой моделью пара документов оказалась в неправильном порядке.

3. **Списочный подход (listwise)**. Его суть заключается в построении модели, на вход которой поступают сразу все документы, соответствующие запросу, а на выходе получается их перестановка.

### Оценка качества

Для оценивания качества ранжирования найденных документов в поиске используются асессорские оценки. Само оценивание происходит на скрытых от обучения запросах $Queries$. Для этого традиционно используется метрика *DCG* ([Discounted Cumulative Gain](https://en.wikipedia.org/wiki/Discounted_cumulative_gain)) и ее нормализованный вариант — *nDCG*, всегда принимающий значения от 0 до 1.
Для одного запроса DCG считается следующим образом:
$$ DCG = \sum_{i=1}^P\frac{(2^{rel_i} - 1)}{\log_2(i+1)}, $$

где $P$ — число документов в поисковой выдаче, $rel_i$ — релевантность (асессорская оценка) документа, находящегося на i-той позиции.     
      
*IDCG* — идеальное (наибольшее из возможных) значение *DCG*, может быть получено путем ранжирования документов по убыванию асессорских оценок.

Итоговая формула для расчета *nDCG*:

$$nDCG = \frac{DCG}{IDCG} \in [0, 1].$$

Чтобы оценить значение *nDCG* на выборке $Queries$ ($nDCG_{Queries}$) размера $N$, необходимо усреднить значение *nDCG* по всем запросам  выборки:
$$nDCG_{Queries} = \frac{1}{N}\sum_{q \in Queries}nDCG(q).$$

Пример реализации метрик ранжирование на python можно найти [здесь](https://gist.github.com/mblondel/7337391).

## Погнали
###  **Задача: предсказать оценку релевантности для запросов тестового датасета**

Мы будем работать на данных с конкурса [Интернет-математика 2009](http://imat2009.yandex.ru/datasets). По ссылке можно прочитать описание данных.      


Данные разбиты на два файла – обучающее множество *imat2009_learning.txt* и множество для оценки *imat2009_test.txt*.   
Файл с обучающим множеством содержит **97 290 строк**, которые соответствуют **9 124 запросам**.   
Каждая строка файлов данных соответствует паре «запрос-документ».     
Каждой паре «запрос-документ» соответствуют значения **245 признаков**. Если значение признака равно 0, то он опускается.     
В комментариях в конце каждой строки указан **идентификатор запроса**.   
Файл с обучающей выборкой содержит **оценку релевантности**, значения из диапазона **[0, 4]** (4 – «высокая релевантность», 0 – «нерелевантно»).   


### DATA

In [1]:
# загрузка данных для обучения
file_learning = './rankings/imat2009_learning.txt'
with open(file_learning) as f:
    lines_learning = f.readlines()
    
# загрузка данных для теста
file_test = './rankings/imat2009_test.txt'
with open(file_test) as f:
    lines_test = f.readlines()

Покрутите данные, посмотрите на их размеры, на то, что лежит внутри 

In [2]:
print(len(lines_learning))
print(len(lines_test))

97290
115643


In [6]:
print("\n- ".join(lines_learning[0:4]))

1 1:0.000023 7:0.704953 8:0.550315 9:0.032294 11:0.712631 14:0.015686 15:0.137255 16:0.302576 17:1.000000 18:0.996078 22:1.000000 23:1.000000 24:1.000000 27:0.700000 28:0.587629 29:0.999881 30:0.032294 34:0.000023 36:0.431373 37:0.002247 38:0.054902 41:1.000000 46:0.002247 50:0.032294 51:0.325613 52:0.056641 53:0.820677 54:0.388235 55:0.450980 56:0.312547 57:0.004672 59:1.000000 61:0.000023 65:1.000000 68:0.712195 69:0.001400 70:1.000000 71:0.001013 73:0.709459 74:0.560784 76:0.142857 77:0.360800 78:1.000000 79:1.000000 80:1.000000 82:0.000023 83:1.000000 85:0.996078 86:0.070588 87:1.000000 88:0.999797 92:1.000000 93:0.714286 95:0.039216 97:0.000023 98:0.356490 99:0.165041 102:1.000000 103:1.000000 104:1.000000 105:0.486275 108:0.152941 120:0.996078 121:0.676507 122:0.032294 126:0.712980 128:0.121569 129:0.609261 132:1.000000 134:0.109804 135:0.030535 140:0.002247 142:0.698039 144:0.248111 145:0.356490 146:1.000000 147:0.498039 148:0.125490 150:0.704953 151:1.000000 152:0.098039 154:0.

Выведите все элементы матрицы для одного запроса

In [ ]:
for line in lines_learning:
    feats_value = line.split(' ')
    ### Your code here



Разбейте обучающую выборку на обучение и контроль в соотношении 70 / 30. Обратите внимание на формат данных: разбивать необходимо **множество запросов**, а не строчки датасета, чтобы в выборке находилась вся информация по запросу.

In [4]:
import numpy as np
from collections import defaultdict

labels = [] #оценки ассесоров
queries = defaultdict(list) #словарь {запрос: индексы строчек датасета}

#матрица данных
data = np.zeros((len(lines_learning), 245), dtype=np.float32) 


In [118]:
# check
assert data.shape == (97290, 245)
assert len(queries.keys()) == 9124
assert len(labels) == 97290

Разделим запросы на обучающую *queries_train* и валидационную выборки *queries_test* (70/30)

In [6]:
from sklearn.model_selection import train_test_split

In [112]:
# check
assert len(queries_train) / (len(queries_train) + len(queries_test)) == 0.6999123191582639

 > Теперь у нас есть:  
 1) айдишники запросов для обучения и валидации **queries_train, queries_test**   
 2) матрица данных **data**   
 3) словарь **queries** с информацией о том, какие строчки в этой матрице соответствуют какому айдишнику  
 
 С помощью этих данных разделите матрицу data на матрицы **X_train, y_train, X_test, y_test**

In [182]:
# напоминание, как можно легко получить данные матрицы по id строки

data[[2, 10, 11]].shape

(3, 245)

In [151]:
train_index, test_index = [], []

### Your code here

X_train, y_train = data[train_index], labels[train_index]
X_test, y_test = data[test_index], labels[test_index]

In [152]:
print(X_train.shape, len(y_train))
print(X_test.shape, len(y_test))

(68418, 245) 68418
(28872, 245) 28872


Данные готовы! Можно приступать к обучению алгоритма.   
Для оценивания качества моделей используйте метрику nDCG, реализованную ниже:

In [ ]:
import metrics


def nDCGQueries_score(all_queries, test_queries, test_index, labels, y_predicted):
    score = [] # nDCG по каждому запросу
    
    for query in test_queries:
        doc_ind = all_queries[query]
        doc_ind_in_testdata = [np.where(test_index==ind)[0][0] for ind in doc_ind]
        nDCG = metrics.ndcg_score(labels[doc_ind], y_predicted[doc_ind_in_testdata], k=len(doc_ind))
        score.append(nDCG)
        
    nDCGQueries = np.sum(score) / len(queries_test) # усредняем по всем запросам
    return nDCGQueries

### FIT PREDICT

**Поточечный подход**    
Воспользовавшись известными вам техниками построения линейной регрессии, обучите модель, предсказывающую оценку асессора.   
> ``` Ex: from sklearn.linear_model import LinearRegression``` 

In [157]:
from sklearn.linear_model import LinearRegression


In [ ]:
score_lin_reg = nDCGQueries_score(queries, queries_test, labels, labels_predicted)
print(score_lin_reg)

Всегда полезно визуализировать данные и результаты, можно увидеть какие-то инсайты или ошибки

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

h = plt.hist(labels_predicted, bins=200)

Давайте теперь решим эту задачу не как регрессию, а как классификацию

In [210]:
from sklearn.svm import LinearSVC

y_train_round = [round(y) for y in y_train]

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)

In [ ]:
score_lin_SVC = nDCGQueries_score(queries, queries_test, test_index, labels, labels_predicted)
print(score_lin_SVC)

h = plt.hist(labels_predicted, bins=10)

#### Ранжируем с XGBoost

In [184]:
import xgboost as xgb
import time


param = {
    'n_estimators': 500,
    'learning_rate': 0.05,
    'max_depth': 5,
    'subsample': 0.7,
    'colsample_bytree': 0.7,
    'objective': 'reg:linear',
    'silent': 1
}


0.8756518399443524


In [ ]:
score_xgb = nDCG_score(queries, queries_test, test_index, labels, labels_predicted)
print(score_xgb)

h = plt.hist(labels_predicted, bins=200)

#### Ранжируем с RandomForestRegressor

In [186]:
from sklearn.ensemble import RandomForestRegressor

n_estimators = 10


elapsed: 4578.151226758957
0.8708130777994284


In [ ]:
score_rdf = nDCGQueries_score(queries, queries_test, test_index, labels, labels_predicted)
print(score_rdf)

h = plt.hist(labels_predicted, bins=200)

![](https://www.safaribooksonline.com/library/view/machine-learning-with/9781785889936/assets/image_03_002.png)

Последняя стадия PREDICTION для нас сейчас неактуальна, поскольку соревнование уже закончилось, и некому оценить предсказание нашего алгоритма на данных без разметки.   
Не в соревновательном ML эта та стадия, когда ваш алгоритм запускают в продакшн, где правильные ответы нигде не прописаны, и его работу оценивают по косвенным признакам.

Стрелочка между TRAINING и TESTING подразумевает подбор оптимальных параметров для алгоритма. Для этого можно использовать простой перебор циклом по списку значений параметра.